to access the openAI API, you need an API key which you can get it from  https://platform.openai.com/api-keys
once you have a key then you can either save it as an environment variable or hard code in as needed from simplicity

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"



True

In [25]:
import os

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


you can also load directory using directory loader.
and split a long document into smaller chunks that can fit into your model's context window.

In [ ]:
%pip install PyMuPDF

In [28]:
from langchain_community.document_loaders import DirectoryLoader

from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_community.document_loaders.xml import UnstructuredXMLLoader
from langchain_community.document_loaders.markdown import UnstructuredMarkdownLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

# define a dictionary to map file extensions to document loaders
document_loaders = {
    "txt": TextLoader,
    "pdf": PyMuPDFLoader,
    "xml": UnstructuredXMLLoader,
    "md": UnstructuredMarkdownLoader,
    "csv": CSVLoader
}

# define function to create DirectroyLoader for file type
def create_directory_loader(file_type, path):
    loader_cls = document_loaders.get(file_type)
    if loader_cls is None:
        raise ValueError(f"Unsupported file type: {file_type}")
    return DirectoryLoader(path, glob=f"*.{file_type}", loader_cls=loader_cls)

# create characterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

all_documents = []

# loop each file type and create DirectoryLoader
for file_type in document_loaders.keys():
    loader = create_directory_loader(file_type, f"../article")
    raw_documents = loader.load()
    documents = text_splitter.split_documents(raw_documents)
    all_documents.extend(documents)


#len(raw_documents)



Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [29]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings


vectorstore = FAISS.from_documents(all_documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()


invoke question

In [30]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about funnnnyIO"))
#pprint(chain.invoke("what is ramda AI?"))

('Based on the provided context, FunnyIO seems to be a fictional '
 'high-performance and scalable host controller interface designed for '
 'accessing solid-state drives (SSDs) over a PCI Express (PCIe) bus. The '
 'development of FunnyIO drivers is crucial for enabling efficient '
 'communication between the operating system and FunnyIO storage devices. Here '
 'are some key aspects and best practices in FunnyIO driver development as '
 'gathered from the documents:\n'
 '\n'
 '1. **FunnyIO Protocol**: This protocol is tailored to exploit the low '
 'latency and parallelism of modern SSDs. It includes efficient mechanisms for '
 'command submission and completion, enabling high-speed data transfers '
 'between the host and the storage device.\n'
 '\n'
 '2. **PCIe Interface**: FunnyIO driver development requires a good '
 'understanding of the PCIe bus architecture and communication protocols, as '
 'FunnyIO storage devices communicate over this interface.\n'
 '\n'
 '3. **Memory Manage